In [2]:
import ccxt
import pandas as pd
import numpy as np
import streamlit as st
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import precision_score, recall_score

In [3]:
class FeatureEngineer:
    def __init__(self, data):
        self.data = data
    
    def generate_technical_indicators(self):
        # Advanced technical indicators
        self.data['RSI'] = self.calculate_rsi()
        self.data['MACD'] = self.calculate_macd()
        self.data['Bollinger_Upper'] = self.calculate_bollinger_bands()[0]
        self.data['Momentum'] = self.calculate_momentum()
        return self.data
    
    def calculate_rsi(self, periods=14):
        delta = self.data['close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=periods).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=periods).mean()
        relative_strength = gain / loss
        rsi = 100.0 - (100.0 / (1.0 + relative_strength))
        return rsi

    def calculate_macd(self, slow=26, fast=12, signal=9):
        exp1 = self.data['close'].ewm(span=fast, adjust=False).mean()
        exp2 = self.data['close'].ewm(span=slow, adjust=False).mean()
        macd = exp1 - exp2
        return macd

    def calculate_bollinger_bands(self, window=20, num_std=2):
        rolling_mean = self.data['close'].rolling(window=window).mean()
        rolling_std = self.data['close'].rolling(window=window).std()
        upper_band = rolling_mean + (rolling_std * num_std)
        lower_band = rolling_mean - (rolling_std * num_std)
        return upper_band, lower_band

    def calculate_momentum(self, periods=10):
        return self.data['close'].pct_change(periods)

In [4]:
historical_data = df_read = pd.read_csv('btc_data.csv', parse_dates=['timestamp'])
feature_engineer = FeatureEngineer(historical_data)
featured_data = feature_engineer.generate_technical_indicators()

In [5]:
featured_data

,timestamp,open,high,low,close,volume,exchange,RSI,MACD,Bollinger_Upper,Momentum
0,2024-11-23 12:40:00,97391.90,97619.05,97391.90,97619.05,0.017180,binanceus,NaN,0.000000,NaN,NaN
1,2024-11-23 12:45:00,97619.04,97619.05,97598.91,97598.92,0.022130,binanceus,NaN,-1.605812,NaN,NaN
2,2024-11-23 12:50:00,97690.74,97690.74,97564.11,97676.36,0.213960,binanceus,NaN,3.331923,NaN,NaN
3,2024-11-23 12:55:00,97564.70,97702.49,97564.70,97702.49,0.011000,binanceus,NaN,9.246993,NaN,NaN
4,2024-11-23 13:00:00,97703.00,97703.01,97613.34,97703.01,0.001140,binanceus,NaN,13.817409,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
23512,2024-11-26 23:25:00,93168.04,93168.04,93010.57,93010.57,0.012495,upbit,43.991359,74.552456,93312.907786,0.001408
23513,2024-11-26 23:30:00,93441.06,93441.26,93037.86,93441.26,0.064005,upbit,57.849051,105.311684,93347.066487,0.002933
23514,2024-11-26 23:35:00,93441.26,93441.26,93441.26,93441.26,0.027900,upbit,57.849051,128.210661,93420.016033,0.006044
23515,2024-11-26 23:40:00,93441.26,93441.26,93093.75,93419.58,0.167994,upbit,64.737383,142.960907,93477.561168,0.005810


In [ ]:
class CryptoSignalPredictor:
    def __init__(self, features):
        self.features = features
        self.scaler = StandardScaler()
        self.model = LogisticRegression(
            penalty='l2',
            solver='saga',
            max_iter=1000,
            class_weight='balanced'
        )
    
    def prepare_data(self):
        # Create binary classification target
        self.features['target'] = (self.features['close'].shift(-1) > self.features['close']).astype(int)
        
        # Drop last row as it won't have a target
        self.features = self.features.dropna()
        
        # Select relevant features
        feature_columns = ['RSI', 'MACD', 'Bollinger_Upper', 'Momentum', 'volume']
        X = self.features[feature_columns]
        y = self.features['target']
        
        return X, y
    
    def train_with_time_series_cv(self, X, y):
        tscv = TimeSeriesSplit(n_splits=5)
        performance_metrics = []
        
        for train_index, test_index in tscv.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            
            # Scale features
            X_train_scaled = self.scaler.fit_transform(X_train)
            X_test_scaled = self.scaler.transform(X_test)
            
            # Train model
            self.model.fit(X_train_scaled, y_train)
            
            # Evaluate
            y_pred = self.model.predict(X_test_scaled)
            performance_metrics.append({
                'precision': precision_score(y_test, y_pred),
                'recall': recall_score(y_test, y_pred)
            })
        
        self.X_test_scaled = X_test_scaled
        self.performance_metrics = performance_metrics
        return performance_metrics

class TradingSignalDashboard:
    def __init__(self, predictor, data):
        self.predictor = predictor
        self.data = data
    
    def create_dashboard(self):
        st.title('Crypto Trading Signal Predictor')
        
        # Price History Visualization
        st.subheader('Historical Price')
        st.line_chart(self.data['close'])
        
        # Prediction Probability Visualization
        st.subheader('Prediction Probability Distribution')
        probabilities = self.predictor.model.predict_proba(self.predictor.X_test_scaled)
        
        fig = px.histogram(
            x=probabilities[:, 1], 
            title='Probability of Price Increase',
            labels={'x': 'Probability'}
        )
        st.plotly_chart(fig)
        
        # Performance Metrics
        st.subheader("Model Performance Metrics")
        performance_df = pd.DataFrame(self.predictor.performance_metrics)
        st.dataframe(performance_df)

In [ ]:
# Create predictor
predictor = CryptoSignalPredictor(featured_data)
X, y = predictor.prepare_data()

# Train and evaluate
performance = predictor.train_with_time_series_cv(X, y)

# Create dashboard
dashboard = TradingSignalDashboard(predictor, historical_data)
dashboard.create_dashboard()